In [2]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 30.2 MB/s eta 0:00:00


In [3]:
import stanza
import math
import random

In [4]:
nlp_mask = stanza.Pipeline('en')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). 

In [5]:
def drop_keyterms(key_terms):
  if len(key_terms) <= 4:
    return key_terms
  percentage_drop = 0.2
  drop_count = math.ceil(len(key_terms) * percentage_drop)
  terms_to_drop = set(random.sample(key_terms, drop_count))

  # Keep the remaining terms in their original order
  remaining_terms = [term for term in key_terms if term not in terms_to_drop]

  return remaining_terms

In [6]:
def extract_terms(question):
  # doc = nlp_mask(question)

  # key_terms = []
  # for sentence in doc.sentences:
  #     for word in sentence.words:
  #         # Example criteria for key terms: nouns, proper nouns, and adjectives
  #         if word.upos in ['NOUN', 'PROPN', 'ADJ', 'ADV', 'VERB']:
  #             key_terms.append(word.text)
  # # print(key_terms)
  # terms_extract = drop_keyterms(key_terms)

  # return terms_extract

  doc = nlp_mask(question)

  # Extract named entities (NER)
  key_terms = [ent.text for ent in doc.ents]

  # Extract compound phrases and adjective-noun pairs using dependency parsing
  for sentence in doc.sentences:
      for word in sentence.words:
          if word.deprel in ['compound', 'amod']:  # compound nouns and adjective modifiers
              # Combine the modifier and the head
              compound_term = f"{word.text} {sentence.words[word.head - 1].text}"
              if compound_term not in key_terms:
                  key_terms.append(compound_term)

  # Add remaining single keywords (nouns, proper nouns, verbs, adverbs)
  for sentence in doc.sentences:
      for word in sentence.words:
          if word.upos in ['NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV']:
              if word.text not in key_terms:
                  key_terms.append(word.text)

  terms_extract = drop_keyterms(key_terms)

  return terms_extract

In [7]:
from openai import OpenAI

In [8]:
client = OpenAI(api_key = "")

In [9]:
def generate_relevant_info_gpt4o(key_terms: list):
  terms = ", ".join(key_terms)

  system_prompt = "You are a helpful assistant that gives relevant information. Provide a comprehensive context or paragraph that meaningfully incorporates the given key terms or words."
  user_prompt = f"""
    Provide relevant information and context incorporating the following key terms.
    Key terms: {terms}
    """

  response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

  completions = response.choices[0].message.content.strip()

  return completions

In [10]:
def judge_response(question, answer, response):
  system_prompt = "You are role is to judge a given task. Provide your judgement in just one word, either TRUE or FALSE."
  user_prompt = f"""
    Can the given idea be entailed from the context?
    Idea: {answer}
    Context: {response}
    """

  response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

  completions = response.choices[0].message.content.strip()

  if completions == "TRUE":
    return True
  elif completions == "FALSE":
    return False
  else:
    return ""

In [11]:
import pandas as pd

In [12]:
df_mmlu = pd.read_csv('/content/custom_mmlu.csv')

df_m = pd.DataFrame()

df_m['question'] = df_mmlu['question']

print(df_m.head())

   Unnamed: 0                                           question  \
0         823  A fluorescent molecule of 1,000 daltons inject...   
1         580    Which of the following is not a true statement?   
2        6106  Surveys of older adults who are living togethe...   
3       11664  Third-year student is studying contracts. He h...   
4       13728  The particular quality of the U.S. health care...   

              subject                                            choices  \
0     college_biology  ['spot desmosome' 'belt desmosome' 'gap juncti...   
1  clinical_knowledge  ['Muscle glycogen is broken down enzymatically...   
2         human_aging  ['A first step toward eventual marriage' 'An a...   
3    professional_law  ["An assignment that continues the obligor's d...   
4            virology  ['Systematic, democratic approach' 'Fragmented...   

   answer  
0       2  
1       3  
2       1  
3       0  
4       1  
                                            question
0  A fluo

In [13]:
def parse_choices(choices_string):
    choices_list = choices_string.strip("[]").split(" '")
    choices_list = [choice.replace("'", "").strip() for choice in choices_list]
    return choices_list

In [14]:
df_m['choices'] = df_mmlu['choices'].apply(parse_choices)

In [15]:
print(df_m.head())

                                            question  \
0  A fluorescent molecule of 1,000 daltons inject...   
1    Which of the following is not a true statement?   
2  Surveys of older adults who are living togethe...   
3  Third-year student is studying contracts. He h...   
4  The particular quality of the U.S. health care...   

                                             choices  
0  [spot desmosome, belt desmosome, gap junction,...  
1  [Muscle glycogen is broken down enzymatically ...  
2  [A first step toward eventual marriage, An alt...  
3  ["An assignment that continues the obligors du...  
4  [Systematic, democratic approach, Fragmented, ...  


In [16]:
df_m['answer'] = None

problematic_rows = []

for index, row in df_m.iterrows():

  try:
        # Access and process the 'choices' and 'answer' columns
        # print("inx: ", index, "row: ", row)
      df_m.at[index, 'answer'] = df_m.at[index, 'choices'][df_mmlu.at[index, 'answer']]
  except Exception as e:
      # Log the index of problematic rows
      print(f"Error at index {index}: {e}")
      problematic_rows.append(index)

# Drop problematic rows after processing all rows
if problematic_rows:
    df_m = df_m.drop(index=problematic_rows).reset_index(drop=True)
    print(f"Dropped rows with indices: {problematic_rows}")

# Display the processed DataFrame
print(df_m.head())

Error at index 17: list index out of range
Error at index 25: list index out of range
Error at index 31: list index out of range
Error at index 67: list index out of range
Error at index 68: list index out of range
Dropped rows with indices: [17, 25, 31, 67, 68]
                                            question  \
0  A fluorescent molecule of 1,000 daltons inject...   
1    Which of the following is not a true statement?   
2  Surveys of older adults who are living togethe...   
3  Third-year student is studying contracts. He h...   
4  The particular quality of the U.S. health care...   

                                             choices  \
0  [spot desmosome, belt desmosome, gap junction,...   
1  [Muscle glycogen is broken down enzymatically ...   
2  [A first step toward eventual marriage, An alt...   
3  ["An assignment that continues the obligors du...   
4  [Systematic, democratic approach, Fragmented, ...   

                                              answer  
0       

In [17]:
df_m['extracted_terms'] = None
df_m['can_be_entailed'] = None

# Loop through each row using iterrows()
for index, row in df_m.iterrows():
  ET = extract_terms(row['question'])
  df_m.at[index, 'extracted_terms'] = ET

  R = generate_relevant_info_gpt4o(ET)

  J = judge_response(row['question'], row['answer'], R)

  df_m.at[index, 'can_be_entailed'] = J

print(df_m.head)

<bound method NDFrame.head of                                              question  \
0   A fluorescent molecule of 1,000 daltons inject...   
1     Which of the following is not a true statement?   
2   Surveys of older adults who are living togethe...   
3   Third-year student is studying contracts. He h...   
4   The particular quality of the U.S. health care...   
..                                                ...   
90  Which of the following is an example of a chem...   
91  Buyer purchases an item from a merchant that t...   
92  Intermediaries assist end-users by bringing a ...   
93  Which step explains how to find the value of a...   
94  Which of the following countries is actively t...   

                                              choices  \
0   [spot desmosome, belt desmosome, gap junction,...   
1   [Muscle glycogen is broken down enzymatically ...   
2   [A first step toward eventual marriage, An alt...   
3   ["An assignment that continues the obligors du...   


In [18]:
df_m.to_csv('relevant_info_mmlu.csv', index=False)